# Basic imports

In [2]:
import pandas as pd
import os
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams.update({'figure.max_open_warning': 0})

In [3]:
current_path = os.getcwd()
root_path=current_path.replace('\\forward_feature_selection','')

# Loading dataframe

In [4]:
mixed_df=pd.read_csv(root_path+"\\molecules.csv",sep="\t")

# Functions

In [5]:
def save_df_to_disk(df,name:str,separator="\t"):
    df.to_csv(name,sep=separator,index = False, header=True)

In [6]:
def get_data_and_true_prediction(df,not_wanted_features:list):
        temp_df=df.drop(not_wanted_features,axis=1)
        y=temp_df[temp_df.columns[-1]]
        x=temp_df.drop([temp_df.columns[-1]],axis=1)
        
        return x,y   

In [7]:
def get_df_with_name_and_prediction(df,true_prediction,big_df):
    new_df=df
    new_df.insert(0,"m_name",big_df["m_name"].values)
    new_df=new_df.join(true_prediction)
    
    return new_df

# Forward selection

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.svm import LinearSVC

In [9]:
def get_roc_auc_score(x,y,model): # gets roc auc average
        cv_results = cross_validate(model, x, y, cv=10,scoring=('roc_auc'))
        roc_auc_avrg=cv_results['test_score'].mean()
        
        return roc_auc_avrg 

In [10]:
def forward_selection(x,y,model): # O(n) worst case scenario, where n depends on len(x.columns)
    
    def first_iteration(x,y,model):
        score_lst=[]
        for i in range(len(x.columns)):
            k=x.columns[i]
            temp_x=x[[k]]
            score=get_roc_auc_score(temp_x,y,model)
            score_lst.append(score)

        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        new_features=[x.columns[i] for i in max_score_index]
        top_new_features=new_features[0]
        best_x=x[top_new_features]
        new_x=x.drop(top_new_features,axis=1)
        
        return best_x,new_x,max_score
    
    def else_iteration(best_x,x,y,model,actual_score):     
        new_x_lenght = len(x.columns)
        if (new_x_lenght > 0):
            score_lst=[]
            for i in range(new_x_lenght):
                k=x.columns[i]
                temp_x=x[[k]]
                temp_new_x=pd.concat([best_x,temp_x],axis=1, ignore_index=True)
                score=get_roc_auc_score(temp_new_x,y,model)
                score_lst.append(score)

            new_max_score = max(score_lst) # best score
            actual_best_score = actual_score # score passed from parameters

            if(new_max_score<actual_best_score):
                return best_x,actual_best_score # break condition, recursive function

            max_score_index=[i for i, j in enumerate(score_lst) if j == new_max_score] # indx with best score 

            new_features=[x.columns[i] for i in max_score_index]
            top_new_features=new_features[0]
            temp_x=x[top_new_features]
            best_x=pd.concat([best_x,temp_x],axis=1)
            new_x=x.drop(top_new_features,axis=1)

            return else_iteration(best_x,new_x,y,model,new_max_score)
        
        return best_x,actual_score
    
    best_x,new_x,score=first_iteration(x,y,model)
    best_x,score=else_iteration(best_x,new_x,y,model,score)
    
    return best_x,score

In [11]:
def forward_partition_lineal(x,y,lst:list,other): # list must be greater than len 2    
    print("\nfirst param list len is {}".format(len(lst)))
    print("first param list -> {}".format(lst))
    
    if (len(lst) == 2):
        first = lst[0]
        print("first is {}".format(first))
        last = lst[1]
        print("last is {}".format(last))
        clf_first = LinearSVC(random_state=0, tol=other,C=first, dual=False) # linear model
        first_df,first_score = forward_selection(x,y,clf_first)
        clf_last = LinearSVC(random_state=0, tol=other,C=last, dual=False) # linear model
        last_df,last_score = forward_selection(x,y,clf_last)
        
        if (last_score > first_score):
            print("last score is greater than first score")
            return last_df,last,last_score
        
        print("first score is greater than last score")
        return first_df,first,first_score
    
    elif(len(lst) == 3):
        first = lst[0]
        print("first is {}".format(first))
        middle = lst[1]
        print("middle is {}".format(middle))
        last = lst[-1]
        print("last is {}".format(last))
        clf_first = LinearSVC(random_state=0, tol=other,C=first, dual=False) # linear model
        _,first_score = forward_selection(x,y,clf_first)       
        clf_middle = LinearSVC(random_state=0, tol=other,C=middle, dual=False) # linear model
        _,middle_score = forward_selection(x,y,clf_middle)
        clf_last = LinearSVC(random_state=0, tol=other,C=last, dual=False) # linear model
        _,last_score = forward_selection(x,y,clf_last)
        
        if ((first_score > middle_score) and (middle_score > last_score)):
            print("sub list taken: left")
            return forward_partition_lineal(x,y,lst[:2],other)  
        
        print("sub list taken: right")            
        return forward_partition_lineal(x,y,lst[1:],other)
    
    else:          
        first = lst[0]
        print("first is {}".format(first))
        middle = lst[(len(lst)//2)-1]
        print("middle is {}".format(middle))
        last = lst[-1]
        print("last is {}".format(last))
        clf_first = LinearSVC(random_state=0, tol=other,C=first, dual=False) # linear model
        _,first_score = forward_selection(x,y,clf_first)       
        clf_middle = LinearSVC(random_state=0, tol=other,C=middle, dual=False) # linear model
        _,middle_score = forward_selection(x,y,clf_middle)
        clf_last = LinearSVC(random_state=0, tol=other,C=last, dual=False) # linear model
        _,last_score = forward_selection(x,y,clf_last)
        
        if ((first_score > middle_score) and (middle_score > last_score)):
            print("sub list taken: left")
            return forward_partition_lineal(x,y,lst[:(len(lst)//2)],other)
        
        print("sub list taken: right")            
        return forward_partition_lineal(x,y,lst[(len(lst)//2)-1:],other)          

In [12]:
def best_lineal(x,y,K:list,P:list):
    
    def get_best_score_index(score_lst:list):
        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        
        return max_score_index[0]
    
    best_prm_lst = []
    for i in P:
        print("Seccond parameter is: {}".format(i))
        best_df,best_k,best_score = forward_partition_lineal(x,y,K,i)      
        print("\nBest first parameter is: {}".format(best_k))
        print("ROC AUC = {}".format(best_score))
        print("\nFeatures\n")
        for c,i in enumerate(best_df.columns):
            print("{}. {}".format(c+1,i))
        best_prm_lst.append([best_df,best_k,i,best_score])
        print("")
        
    score_lst = [d  for [a,b,c,d] in best_prm_lst]
    index = get_best_score_index(score_lst)
    print("Final results")
    print("Best model when first param = {} ,second param = {} ,ROC AUC = {}".format(best_prm_lst[index][1],
                                                                 best_prm_lst[index][2],
                                                                 best_prm_lst[index][3]))
    return best_prm_lst[index][0]

In [17]:
def guarantee_results(x,y,K:list,P:list):
    
    def get_best_score_index(score_lst:list):
        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        
        return max_score_index[0]
    
    best_outer_prm_lst = []
    for i in K:
        print("First param is {}\n".format(i))
        best_inner_prm_lst = []
        for j in P:
            print("Second param is {}".format(j))
            clf = LinearSVC(random_state=0, tol=j,C=i, dual=False) # linear model
            best_df,best_score = forward_selection(x,y,clf)
            print("ROC AUC = {}".format(best_score))
            print("\nFeatures\n")
            for c,d in enumerate(best_df.columns):
                print("{}. {}".format(c+1,d))       
            print("")
            best_inner_prm_lst.append([best_df,i,j,best_score])
            
        score_lst = [d  for [a,b,c,d] in best_inner_prm_lst]
        index = get_best_score_index(score_lst)
        best_outer_prm_lst.append([best_inner_prm_lst[index][0],best_inner_prm_lst[index][1],best_inner_prm_lst[index][2],best_inner_prm_lst[index][3]])
        
    score_lst = [d  for [a,b,c,d] in best_outer_prm_lst]
    index = get_best_score_index(score_lst)
    print("Final results")
    print("Best model when first param = {} ,second param = {} ,ROC AUC = {}".format(best_outer_prm_lst[index][1],
                                                                 best_outer_prm_lst[index][2],
                                                                 best_outer_prm_lst[index][3]))
    return best_outer_prm_lst[index][0]

# Data

In [14]:
unnecesary_features=["m_name"]
x,y = get_data_and_true_prediction(mixed_df,unnecesary_features)

In [13]:
C = [1,2,3,4,5,6,7,8,9,10]
tol = [1e-6,1e-5,1e-4,1e-3,1e-2]

In [14]:
start = time.time()
best_df = best_lineal(x,y,C,tol)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

Seccond parameter is: 1e-06

first param list len is 10
first param list -> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
first is 1
middle is 5
last is 10
sub list taken: right

first param list len is 6
first param list -> [5, 6, 7, 8, 9, 10]
first is 5
middle is 7
last is 10
sub list taken: right

first param list len is 4
first param list -> [7, 8, 9, 10]
first is 7
middle is 8
last is 10
sub list taken: right

first param list len is 3
first param list -> [8, 9, 10]
first is 8
middle is 9
last is 10
sub list taken: right

first param list len is 2
first param list -> [9, 10]
first is 9
last is 10
first score is greater than last score

Best first parameter is: 9
ROC AUC = 0.8894640482797351

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_aliphatic_carbocycles
11. n_HOH
12. n_non_strict_rotable_bonds
13. n_saturated_carbocycles
14. n_valence_electrons
15. n_atoms_with

'00:24:00'

In [15]:
k_pivot = 8
p_pivot = 0.001
new_K = [i for i in range(k_pivot-3,k_pivot+4)]
new_P = [0.001,0.01,1]

In [18]:
start = time.time()
g_df = guarantee_results(x,y,new_K,new_P)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

First param is 5

Second param is 0.001
ROC AUC = 0.8868085751036453

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_aliphatic_carbocycles
11. n_HOH
12. n_non_strict_rotable_bonds

Second param is 0.01
ROC AUC = 0.8875769597641346

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_rings
11. n_HOH
12. n_non_strict_rotable_bonds
13. n_saturated_carbocycles
14. n_atoms_without_Hydrogen

Second param is 1
ROC AUC = 0.8515891339626382

Features

1. n_HBD
2. n_non_strict_rotable_bonds
3. n_aliphatic_heterocycles
4. n_HBA
5. n_aromatic_carbocycles
6. n_hetero_cycles
7. n_aromatic_heterocycles
8. n_amide_bonds
9. n_briged_head_atoms
10. n_saturated_carbocycles
11. fraction_CSP3

First param is 6

Second param is 0.001
ROC AUC = 0.89010886374

'00:05:53'

In [19]:
new_df=get_df_with_name_and_prediction(g_df,y,mixed_df)
save_df_to_disk(new_df,"best_classifier_lineal.csv")

C = 8
Tol = 0.001
ROC AUC = 0.8904

In [20]:
new_df.head()

,m_name,n_HBD,n_O,n_aliphatic_heterocycles,m_logp,n_aromatic_rings,n_amide_bonds,n_briged_head_atoms,n_strict_rotable_bonds,n_saturated_rings,n_aliphatic_rings,n_HOH,n_non_strict_rotable_bonds,n_valence_electrons,n_atoms_without_Hydrogen,n_aromatic_carbocycles,n_saturated_carbocycles,fraction_CSP3,is_cns_molecule
0,BUMETANIDE,3,7,0,3.0365,2,0,0,8,0,0,4,8,134,25,2,0,0.235294,0
1,BACLOFEN,2,3,0,1.8570,1,0,0,4,0,0,3,4,76,14,1,0,0.300000,1
2,METYRAPONE,0,3,0,2.6371,2,0,0,3,0,0,0,3,86,17,0,0,0.214286,0
3,METHYLPHENOBARBITAL,1,5,1,1.0426,1,4,0,2,1,1,1,2,94,18,1,0,0.307692,1
4,DULOXETINE,1,2,0,4.6309,3,0,0,6,0,0,1,6,108,21,2,0,0.222222,1
